# HW1 布尔检索

本次作业使用斯坦福大学[CS 276 / LING 286: Information Retrieval and Web Search](https://web.stanford.edu/class/cs276/)课程的代码框架来实现。具体来说主要包含的内容有：
1. [索引构建 (40%)](#索引构建与检索-(40%)) 使用BSBI方法模拟在内存不足的情况下的索引构建方式，并应用于布尔查询
2. [索引压缩 (30%)](#索引压缩-(30%)) 使用可变长编码对构建的索引进行压缩
3. [布尔检索 (10%)](#布尔联合检索-(10%)) 对空格分隔的单词查询进行联合（与）布尔检索
3. [实验报告 (10%)](#Report-(25%)) 描述你的代码并回答一些问题
4. [额外的编码方式 (10%)](#额外的编码方式-(10%)) 鼓励使用额外的编码方式对索引进行压缩 (例如, gamma-encoding)

In [1]:
# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit 
import contextlib
import jieba
import re


In [2]:
print("1")

1


# 数据集

实验使用的文本数据是stanford.edu域下的网页内容，可从http://web.stanford.edu/class/cs276/pa/pa1-data.zip 下载。以下代码将大约170MB的文本数据下载到当前目录下，

In [3]:
import urllib.request
import zipfile

#data_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
data_dir = 'pa1-data'
#urllib.request.urlretrieve(data_url, data_dir+'.zip')
zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
zip_ref.extractall()
zip_ref.close()

之后构建的索引会被存储到`output_dir`，`tmp`会存储测试数据（toy-data）所生成的一些临时文件

In [4]:
try: 
    os.mkdir('output_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('toy_output_dir')
except FileExistsError:
    pass

在数据目录下有10个子目录（命名0-9）

In [5]:
sorted(os.listdir('pa1-data'))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

每一个子目录下的文件都包含一个独立网页的内容。可以认为在同一子目录下没有同名文件，即每个文件的绝对路径不会相同。

In [6]:
sorted(os.listdir('pa1-data/0'))[:10]

['3dradiology.stanford.edu_',
 '3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html',
 '3dradiology.stanford.edu_patient_care_case_studies.html',
 '5-sure.stanford.edu_',
 '50years.stanford.edu_',
 'a3cservices.stanford.edu_awards_nominate_',
 'a3cservices.stanford.edu_facilities_',
 'a3cservices.stanford.edu_lead_',
 'aa.stanford.edu_',
 'aa.stanford.edu_about_aviation.php']

所有的网页内容已经经过处理，仅包含由空格分隔开的单词，不再需要进行额外的标准化工作。

In [7]:
with open('pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
    print(f.read())

3d radiology lab stanford university school of medicine stanford school of medicine 3d and quantitative imaging in the department of radiology search this site only stanford medical sites ways to give find a person alumni lane library ways to give find a person about us mission to develop and apply innovative techniques for efficient quantitative analysis and display of medical imaging data through interdisciplinary collaboration goals education to train physicians and technologists locally and worldwide in the latest developments in 3d and quantitative imaging research to develop new approaches to the exploration analysis and quantitative assesment of diagnostic images that result in a new and or more cost effective diagnostic approaches and b new techniques for the design and planning and monitoring of therapy patient care to deliver valid clinically relevant visualization and analysis of medical imaging data to the stanford community locations richard m lucas magnetic resonance imag

作业目录下有一个小的数据集文件夹`toy-data`。在使用完整网页数据集之前，我们会用它来测试我们的代码是否正确。

In [8]:
toy_dir = 'toy-data'

# 索引构建与检索 (40%)

作业的第一部分是使用**blocked sort-based indexing (BSBI)** 算法来构建倒排索引并实现布尔检索。关于BSBI算法可以参考老师课件或者斯坦福教材[Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf)。以下摘自教材内容

> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

对于无法在内存一次性处理的较大数据集，将会使用到二级存储（如：磁盘）。

## IdMap

再次引用教材 Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

我们首先定义一个辅助类`IdMap`，用于将字符串和数字ID进行相互映射，以满足我们在term和termID、doc和docID间转换的需求。

实现以下代码中的`_get_str` 和 `_get_id`函数，IdMap类的唯一接口是`__getitem__`，它是一个特殊函数，重写了下标运算`[]`,根据下标运算键的类型得到正确的映射值（如果不存在需要添加）。（特殊函数可参考[官方文档](https://docs.python.org/3.7/reference/datamodel.html#special-method-names)）
<br>
<br>
我们会用到字典来将字符串转换为数字，用列表来将数字转换为字符串。

In [9]:
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        return self.id_to_str[i]
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). S
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        for key in  self.str_to_id:
            if key==s:
                return self.str_to_id[s]
        else:
            t=len(self.str_to_id)
            self.str_to_id[s]=t
            self.id_to_str.append(s)
            return t
            

        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

确保代码能通过以下简单测试样例

In [10]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
print(testIdMap.str_to_id)
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

{'a': 0, 'bcd': 1}


之后会需要你自己来写测试样例来确保你的程序正常运行

## 将倒排列表编码成字节数组

为了高效地从磁盘读写倒排列表（文档ID），我们将其存储为字节数组的形式。代码提供了`UncompressedPostings`类来用静态函数实现对倒排列表的编码和解码。在之后的任务中你需要使用该接口实现索引压缩版本（可变长编码）。

参考:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [11]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes
        
        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)
            
        Returns
        -------
        bytes
            bytearray representing integers in the postings_list
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        
        encoded_postings_list: bytes
            bytearray representing encoded postings list as output by encode 
            function
             
        Returns
        -------
        List[int]
            Decoded list of docIDs from encoded_postings_list
            
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

运行以下代码查看其工作方式

In [12]:
x = UncompressedPostings.encode([1,2,3,4,5,6,7])
print(x)

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x04\x00\x00\x00\x05\x00\x00\x00\x06\x00\x00\x00\x07\x00\x00\x00'


## 磁盘上的倒排索引

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 

在这一部分我们提供了一个基类`InvertedIndex`，之后会在此基础上构建它的子类`InvertedIndexWriter`, `InvertedIndexIterator` 和 `InvertedIndexMapper`。在Python中我们常用`cPickle`进行序列化，但是它并不支持部分读和部分写，无法满足BSBI算法的需要，所以我们需要定义自己的存储方式。

In [13]:
class InvertedIndex:
    """A class that implements efficient reads and writes of an inverted index 
    to disk
    
    Attributes
    ----------
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        This is a dictionary that maps from termIDs to a 3-tuple of metadata 
        that is helpful in reading and writing the postings in the index file 
        to/from disk. This mapping is supposed to be kept in memory. 
        start_position_in_index_file is the position (in bytes) of the postings 
        list in the index file
        number_of_postings_in_list is the number of postings (docIDs) in the 
        postings list
        length_in_bytes_of_postings_list is the length of the byte 
        encoding of the postings list
    
    terms: List[int]
        A list of termIDs to remember the order in which terms and their 
        postings lists were added to index. 
        
        After Python 3.7 we technically no longer need it because a Python dict 
        is an OrderedDict, but since it is a relatively new feature, we still
        maintain backward compatibility with a list to keep track of order of 
        insertion. 
    """
    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        Parameters
        index_name (str): Name used to store files related to the index
        postings_encoding: A class implementing static methods for encoding and 
            decoding lists of integers. Default is None, which gets replaced
            with UncompressedPostings
        directory (str): Directory where the index files will be stored
        """

        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')
        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory
        self.postings_dict = {}
        self.terms = []         #Need to keep track of the order in which the 
                                #terms were inserted. Would be unnecessary 
                                #from Python 3.7 onward

    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Load the postings dict and terms from the metadata file
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()
            

        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()
        
        # Write the postings dict and terms to the metadata file
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

因为是在与磁盘上的文件进行交互，所以我们提供了`__enter__`和`__exit__`函数，它使得我们能够像使用python中文件IO一样使用`with`语句。（参考[上下文管理器官方文档](https://docs.python.org/3/library/contextlib.html)）

以下是使用 `InvertedIndexWriter` 上下文管理器的实例:

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

## 索引

> BSBI (i) segments the collection into parts of equal size, (ii) sorts the termID-docID pairs of each part in memory, (iii) stores intermediate sorted results on disk, and (iv) merges all intermediate results into the final index. 

你需要将每一个子目录当做一个块（block），并且在构建索引的过程中每次只能加载一个块（模拟内存不足）。注意到我们是将操作系统意义上的块进行了抽象。你可以认为每个块足够小，能被装载进内存。

在这一部分，我们将阶段性地构造类`BSBIIndex`。函数`index`给出了BSBI算法的框架，而你的工作则是在接下来的部分中实现函数`parse_block`, `invert_write` 和 `merge`。

In [14]:
# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """ 
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg 
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index
    postings_encoding: Encoding used for storing the postings.
        The default (None) implies UncompressedPostings
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", 
                 postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # Stores names of intermediate indices
        self.intermediate_indices = []
        
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
            
    
    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """Base indexing code
        
        This function loops through the data directories, 
        calls parse_block to parse the documents
        calls invert_write, which inverts each block and writes to a new index
        then saves the id maps and calls merge on the intermediate indices
        """
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):#将文件中的子目录排序
            
            td_pairs = self.parse_block(block_dir_relative) #切分块
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id) 
            with InvertedIndexWriter(index_id, directory=self.output_dir, 
                                     postings_encoding=
                                     self.postings_encoding) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id, 
                                          directory=self.output_dir, 
                                          postings_encoding=
                                          self.postings_encoding)) 
                 for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)
               

### 解析

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

你需要将每一个子目录当做一个块，`parse_block`接收子目录路径作为参数。同一子目录下所有文件名都是不同的。

_注意 - 我们使用 `BSBIIndex` 继承 `BSBIIndex`, 这只是对一个已存在类添加新内容的简单方法。在这里只是用来切分类的定义（jupyter notebook内教学使用，无特殊含义）。_

In [15]:
class BSBIIndex(BSBIIndex):            
    def parse_block(self, block_dir_relative):
        """Parses a tokenized text file into termID-docID pairs
        
        Parameters
        ----------
        block_dir_relative : str
            Relative Path to the directory that contains the files for the block
        
        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
        
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        These persist across calls to parse_block
        """
        ### Begin your code
        l=['\n','?','.',':','"','[',']','(',')',' ',',','\'']
        a=os.path.join(self.data_dir,block_dir_relative)
        n=list()
        for i in os.listdir(a):
            #h为文档id
            k=os.path.join(block_dir_relative,i)
            h=self.doc_id_map[k]
            q=os.path.join(a,i)
            with open(q,'r+') as f:
                e=f.read()
                tokens = e.split()
                for j in tokens:
                    if j not in l:
                        n.append((self.term_id_map[j],h))
        return n
                    
                    
                
        ### End your code

In [16]:
BSBI_example = BSBIIndex(data_dir="", output_dir = 'test1')
p=BSBI_example.parse_block("test1")
print(p)

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (1, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (1, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (1, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (9, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0), (11, 0), (37, 0), (36, 0), (38, 0), (39, 0), (40, 0), (41, 0), (42, 0), (43, 0), (44, 0), (45, 0), (46, 0), (47, 0), (48, 0), (49, 0), (50, 0), (48, 0), (51, 0), (52, 0), (53, 0), (54, 0), (55, 0), (16, 0), (21, 0), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (56, 0), (57, 0), (58, 0), (7, 0), (59, 0), (60, 0), (61, 0), (62, 0), (63, 0), (22, 0), (64, 0), (65, 0), (66, 0), (67, 0), (65, 0), (68, 0), (69, 0), (70, 0), (71, 0), (31, 0), (7, 0), (72, 0), (73, 0), (74, 0), (75, 0), (76, 0), (77, 0), (62, 0), (78, 0), (59, 0), 

观察函数在测试数据上是否正常运行

In [17]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



In [18]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(type(f.read()))

<class 'str'>


In [240]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
n=BSBI_instance.parse_block('0')


写一些测试样例来确保`parse_block`方法正常运行（如：相同单词出现时是相同ID）

In [20]:
### Begin your code
BSBI_instance.term_id_map['you']
print(BSBI_instance.term_id_map.str_to_id)
### End your code

{"i'm": 0, 'fine': 1, 'thank': 2, 'you': 3, 'contact': 4, 'us': 5, 'about': 6, 'food': 7, 'allergies': 8, 'stanford': 9, 'medicine': 10, 'school': 11, 'of': 12, 'allergy': 13, 'program': 14, 'search': 15, 'this': 16, 'site': 17, 'only': 18, 'medical': 19, 'sites': 20, 'ways': 21, 'to': 22, 'give': 23, 'find': 24, 'a': 25, 'person': 26, 'alumni': 27, 'lane': 28, 'library': 29, 'for': 30, 'your': 31, 'information': 32, 'doube': 33, 'blind': 34, 'placebo': 35, 'control': 36, 'challenge': 37, 'q': 38, '&a': 39, 'what': 40, 'expect': 41, 'informational': 42, 'video': 43, 'speech': 44, 'from': 45, 'peanut': 46, 'allergic': 47, 'family': 48, 'interested': 49, 'in': 50, 'participating': 51, 'clinical': 52, 'trial': 53, 'please': 54, 'safar': 55, 'scheduler': 56, 'gmail': 57, 'com': 58, 'by': 59, 'email': 60, 'download': 61, 'pre': 62, 'screening': 63, 'form': 64, 'and': 65, 'attach': 66, 'questionaire': 67, 'studies': 68, 'make': 69, 'donation': 70, 'donate': 71, 'now': 72, 'more': 73, 'suppor

### 倒排表

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

在这一部分我们添加函数`invert_write`来实现由termID-docID对构建倒排表。 

但是，我们首先需要实现类`InvertedIndexWriter`。和列表类似，该类提供了append函数，但是倒排表不会存储在内存中而是直接写入到磁盘里。

In [21]:
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')              
        return self

    def append(self, term, postings_list):
        """Appends the term and postings_list to end of the index file.
        
        This function does three things, 
        1. Encodes the postings_list using self.postings_encoding
        2. Stores metadata in the form of self.terms and self.postings_dict
           Note that self.postings_dict maps termID to a 3 tuple of 
           (start_position_in_index_file, 
           number_of_postings_in_list, 
           length_in_bytes_of_postings_list)
        3. Appends the bytestream to the index file on disk

        Hint: You might find it helpful to read the Python I/O docs
        (https://docs.python.org/3/tutorial/inputoutput.html) for
        information about appending to the end of a file.
        
        Parameters
        ----------
        term:
            term or termID is the unique identifier for the term
        postings_list: List[Int]
            List of docIDs where the term appears
        """
        ### Begin your code
        postings_list.sort()
        x= self.postings_encoding.encode(postings_list)
        self.terms.append(term)
        if len(self.postings_dict)==0:
            self.postings_dict[term]=(0,len(postings_list),len(x))
        else:
            o=self.terms[len(self.terms)-2]
            l=self.postings_dict[o]
            k=l[0]+l[2]
            n=len(self.postings_dict)+1
            self.postings_dict[term]=(k,len(postings_list),len(x))
        #ab追加二进制数据
        with open(self.index_file_path, 'ab') as q:
            q.write(x) 

        ### End your code

尽管还没有实现读取索引的类，我们还是可以用以下测试代码检测我们的实现。

In [22]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(0, [3, 2, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [0,2], "terms sequence incorrect"
    assert index.postings_dict == {0: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))} ,"postings_dict incorrect"
    
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"
    

现在我们实现 `invert_write`，它将解析得到的td_pairs转换成倒排表，并使用`InvertedIndexWriter` 类将其写入磁盘。

In [23]:
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs, index):
        """Inverts td_pairs into postings_lists and writes them to the given index
        
        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index on disk corresponding to the block       
        """
        ### Begin your code
        
        #排序
        td_pairs.sort(key = lambda x : x[0])
        #构建postings list,并保存
        k=list()
        key=-1
        for i in td_pairs:
            if key!=-1:
                if key==i[0]:
                    k.append(i[1])
                else:
                    k=list(set(k))       #去重并从小到大排序
                    index.append(key,k)
                    k=[]
                    key=i[0]
                    k.append(i[1])
            else:
                key=i[0]
                k.append(i[1])
        if len(k)!=0:
            k.sort()
            index.append(key,k)
            
        ### End your code

我们可以在测试数据上读取一个块并观察倒排索引中包含的内容。
仿照`InvertedIndexWriter`部分写一些测试样例。

In [24]:
### Begin your code
with InvertedIndexWriter('test' ,directory='tmp/') as index:
    td_pairs=[(0,3),(2,3),(1,4),(4,5),(1,3),(0,1),(4,1)]
    b=BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
    b.invert_write( td_pairs,index) 
    assert index.terms == [0,1,2,4],'incorrect'
    print(index.postings_dict)
### End your code

{0: (0, 2, 8), 1: (8, 2, 8), 2: (16, 1, 4), 4: (20, 2, 8)}


In [25]:
o=[2,1,3,2,4,2,1,5,1,3,2,5,1,2,3]
o=list(set(o))
print(o)

[1, 2, 3, 4, 5]


### 合并
> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

Python中的迭代模型非常自然地符合我们维护一个小的读缓存的要求。我们可以迭代地从磁盘上每次读取文件的一个倒排列表。我们通过构建`InvertedIndex`的子类`InvertedIndexIterator`来完成这个迭代任务。

In [26]:
class InvertedIndexIterator(InvertedIndex):
    """"""
       
    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """Use this function to initialize the iterator
        """
        ### Begin your code
        self.k=0
        self.t=0
        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):
        """Returns the next (term, postings_list) pair in the index.
        
        Note: This function should only read a small amount of data from the 
        index file. In particular, you should not try to maintain the full 
        index file in memory.
        """
        ### Begin your code
        if len(self.terms)>self.k:
            a=self.terms[self.k]
            t=self.postings_dict[a]
            length=t[2]
            p=t[0]
            self.index_file.seek(p,0)
            h=self.index_file.read(length)
            n=self.postings_encoding.decode(h) #获得posting_list
            self.k+=1
            return (a,n)
        else:
            if self.t==0:
                self.t=1
                return self.k
            else:
                raise StopIteration
    
        
        ### End your code

    def delete_from_disk(self):
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)


为了测试以上代码，我们先用`InvertedIndexWriter` 创建索引，然后再迭代遍历。写一些小的测试样例观察它是否正常运行。至少你得写一个测试，手工构建一个小的索引，用`InvertedIndexWriter`将他们写入磁盘，然后用`InvertedIndexIterator`迭代遍历倒排索引。

In [242]:
### Begin your code
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    with InvertedIndexIterator('test', directory='tmp/') as k:
        print(next(k))
        print(next(k))
    
        
### End your code

(1, [2, 3, 4])
(2, [3, 4, 5])


In [28]:
### Begin your code
path='test'
index_list=[]
for i in os.listdir(path):
    index_id=os.path.join(path,i)
    index_list.append(index_id)
    with InvertedIndexWriter('tt',directory=index_id) as index:
        index.append(1, [2, 3, 4])
        index.append(2, [3, 4, 5])
with contextlib.ExitStack() as stack:
    indices = [stack.enter_context(
        InvertedIndexIterator('tt', 
                              directory=index_id)) for index_id in index_list]
    for i in indices:
        for j in i:
            if isinstance(j,tuple):
                print(j)
### End your code

    

(1, [2, 3, 4])
(2, [3, 4, 5])
(1, [2, 3, 4])
(2, [3, 4, 5])
(1, [2, 3, 4])
(2, [3, 4, 5])


> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

我们将使用`InvertedIndexIterator`来完成读取的部分，用`InvertedIndexWriter`来将合并后的索引写入磁盘。

注意到我们之前一直使用`with` 语句来打开倒排索引文件，但是现在需要程序化地完成这项工作。可以看到我们在基类`BSBIIndex`的`index`函数中使用了`contextlib`([参考文档](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack))
你的任务是合并*打开的*`InvertedIndexIterator`对象（倒排列表），并且通过一个`InvertedIndexWriter`对象每次写入一个文档列表。

既然我们知道文档列表已经排过序了，那么我们可以在线性时间内对它们进行合并排序。事实上`heapq`([参考文档](https://docs.python.org/3.0/library/heapq.html)) 是Python中一个实现了堆排序算法的标准模板。另外它还包含一个实用的函数`heapq.merge`，可以将多个已排好序的输入（可迭代）合并成一个有序的输出并返回它的迭代器。它不仅可以用来合并倒排列表，也可以合并倒排索引词典。

为了让你快速上手`heapq.merge`函数，我们提供了一个简单的使用样例。给出两个以动物和树平均寿命排序的列表，我们希望合并它们。

In [29]:
import heapq
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Giraffe', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

('Giraffe', 28)
('Rhinoceros', 40)
('Giraffe', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Box turtle', 123)
('Bald Cypress', 600)


注意使用`*`将`lifespan_lists`解包作为参数，并且使用`lambda`函数来给出进行排序的key。如果你对它们不熟悉可以参考文档([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions))。

In [30]:
import heapq
import copy
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """Merges multiple inverted indices into a single index
        
        Parameters
        ----------
        indices: List[InvertedIndexIterator]
            A list of InvertedIndexIterator objects, each representing an
            iterable inverted index for a block
        merged_index: InvertedIndexWriter
            An instance of InvertedIndexWriter object into which each merged 
            postings list is written out one at a time
        """
        ### Begin your code
        x=list()
        o=list()
        #o为标签列表
        for i in range(len(indices)):
            o.append(0)
        while sum(o)!=len(indices):
            for i in range(len(indices)):
                if o[i]==0:
                    h=next(indices[i])
                    if isinstance(h,tuple):
                        x.append(h)
                    else:
                        o[i]=1
            x.sort(key=lambda y : y[0])
            n=list()
            x_x=list()
            for k in x:
                if k[0]==x[0][0]:
                    n.extend(k[1])
                else:
                    x_x.append(k)
            n=list(set(n))
            n.sort()
            merged_index.append(x[0][0],n)
            x=x_x
        t=0
        p=list()
        for i in x:
            if t==0:
                t=i
                p.extend(i[1])
            if t[0]==i[0]:
                p.extend(i[1])
            else:
                p=list(set(p))
                p.sort()
                merged_index.append(t[0],p)
                t=i
                p=list()
                p.extend(i[1])
        p=list(set(p))
        p.sort()
        merged_index.append(t[0],p)
        
                
                
                
            
            
            

        ### End your code

首先确保它在测试数据上正常运行

接下来对整个数据集构建索引

In [31]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir ='output_dir', )
#BSBI_instance.index()


In [32]:
c_BSBI_instance1 = BSBIIndex(data_dir='toy-data',output_dir ='toy_output_dir/', )
c_BSBI_instance1.index()


In [33]:
with InvertedIndexIterator('BSBI',directory='toy_output_dir/') as k:
    print(next(k))
    print(next(k))
    print(next(k))
    print(next(k))

(0, [0, 5])
(1, [0, 5])
(2, [0, 5])
(3, [0, 1, 2, 3, 4, 5, 6])


如果你在合并阶段出现了错误，可以利用以下代码来debug。

In [34]:
"""BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)"""

"BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )\nBSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]\nwith InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:\n    with contextlib.ExitStack() as stack:\n        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]\n        BSBI_instance.merge(indices, merged_index)"

## 布尔联合检索 (10%)

我们将实现BSBIIndex中的`retrieve`函数，对于给定的包含由空格分隔tokens的字符串查询，返回包含查询中所有tokens的文档列表。但是我们并不能迭代遍历整个索引或者将整个索引加载到内存来寻找相应的terms（索引太大）。

首先我们要实现`InvertedIndex`的子类`InvertedIndexMapper`，它能够找到对应terms在索引文件中位置并取出它的倒排记录表。

In [35]:
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`.
        
        This function should not iterate through the index file.
        I.e., it should only have to read the bytes from the index file
        corresponding to the postings list for the requested term.
        """
        ### Begin your code
        BSBI_instance = BSBIIndex('', self.directory, )
        BSBI_instance.load()
        h=1
        for i in BSBI_instance.term_id_map.str_to_id.keys():
            if term==i:
                h=0
        if h:
            return []
        k=BSBI_instance.term_id_map[term]
        h=self.postings_dict[k]
        self.index_file.seek(h[0])
        l=self.index_file.read(h[2])
        x=self.postings_encoding.decode(l)
        return x
        ### End your code

In [36]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir ='output_dir', )
#BSBI_instance.index()


写一些测试样例检测`_get_postings_list`的实现

In [37]:
with InvertedIndexMapper('BSBI',directory='toy_output_dir') as k:
        print(k['you'])

[0, 1, 2, 3, 4, 5, 6]


现在我们获得了查询中terms对应的倒排记录表，接着需要求他们的交集。这部分与我们之前作业的merge方法类似，请实现`sorted_intersect`函数，遍历两个有序列表并在线性时间内合并。

In [38]:
import copy
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable]
        Sorted intersection        
    """
    ### Begin your code
    list3=[x for x in list1 if x in list2]
    list3=list(set(list3))
    list3.sort()
    return list3
    
    ### End your code

简单的测试样例

In [39]:
### Begin your code
a=[1,2,3]
b=[3,2,4]
h=sorted_intersect(a,b)
print(h)
### End your code

[2, 3]


现在可以利用`sorted_intersect` 和 `InvertedIndexMapper`来实现`retrieve`函数。

In [47]:
#%%tee submission/retrieve.py
class BSBIIndex(BSBIIndex):
    def retrieve(self, query):
        """Retrieves the documents corresponding to the conjunctive query
        Parameters
        ----------
        query: str
            Space separated list of query tokens
        Result
        ------
        List[str]
            Sorted list of documents which contains each of the query tokens. 
            Should be empty if no documents are found.
        Should NOT throw errors for terms not in corpus
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()
        ### Begin your code
        k=query.split()
        o=list()
        h=list()
        with InvertedIndexMapper('BSBI',postings_encoding=self.postings_encoding, directory=self.output_dir,) as h:
            for i in k:
                if len(h[i])!=0:
                    o.append(h[i])
                else:
                    return []
        p=list()
        if len(o)==1:
            for i in o[0]:
                p.append(self.doc_id_map[i])
            return p
        else:
            for k in range(len(o)):
                if k==0:
                    h=o[k]
                else:
                    h=sorted_intersect(h,o[k])
        for i in h:
            p.append(self.doc_id_map[i])
        return p
                    
                
    
        ### End your code

In [48]:
BSBI_instance1 = BSBIIndex(data_dir='toy-data', output_dir = 'toy_output_dir', )
BSBI_instance1.index()


通过一个简单的查询来测试索引在真实数据集上是否正常工作。

In [49]:
BSBI_instance1.retrieve('you')

['0\\fine.txt',
 '0\\foodallergies.stanford.edu_about_us_contact_us.html',
 '0\\hello.txt',
 '1\\bye.txt',
 '1\\byebye.txt',
 '2\\fine.txt',
 '2\\hello.txt']

In [50]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )

In [51]:
#BSBI_instance.retrieve('we are')

你也可以通过读取文件来测试其中的页面是否真的包含了查询的terms

In [52]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

cs276 information retrieval and web search cs 276 ling 286 information retrieval and web search spring 2011 pandu nayak and prabhakar raghavan lecture 3 units tu th 4 15 5 30 gates b03 tas sonali aggarwal ivan cui valentin spitkovsky and sandeep sripada staff e mail cs276 spr1011 staff lists stanford edu lectures are also available online and on television through scpd sitn course description basic and advanced techniques for text based information systems efficient text indexing boolean and vector space retrieval models evaluation and interface issues web search including crawling link based algorithms and web metadata text web clustering classification text mining syllabus additional information staff contact information piazzza we strongly recommend students to post questions to the course page on www piazzza com instead of sending emails this forum enables students to discuss the questions they encounter in lectures or assignments here is a quick introduction video email if you hav

测试dev queries（提前构建好的查询与结果）是否正确

In [54]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert set(my_results) == set(reference_results), "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


In [55]:
print(type(my_results))

<class 'list'>


如果出现了错误，可以通过以下代码比较输出与正确结果的差异

In [56]:
print(len(my_results))

4232


In [57]:
print(len(reference_results))

4232


In [58]:
[1,2,3]==[3,2,1]

False

确保你构建自己的查询来测试所有可能的边界情况，例如数据集中没有出现的terms，或者拖慢合并的频繁出现的terms。

# 索引压缩  (30%)

在这部分，你将使用可变长字节编码对索引进行压缩。（gap-encoding可选，对gap进行编码）

下面几个Python运算符可能对你有帮助

In [59]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

10 % 2 =  0
10 % 3 =  1
10 / 3 =  3.3333333333333335
10 // 3 =  3


完成下面的`CompressedPostings`类，我们将用它来替换`UncompressedPostings`。关于如何使用gap-encoding和可变长字节编码，你可以参考老师课件或者[Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf)。

In [60]:
import copy
class CompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code
    @staticmethod
    def switch(a):
        m=bytearray()
        if a==0:
            m.append(128)
            return m
        while(a>0):
            b=a%128
            if a<128:
                m.append(b+128)
            else:
                m.append(b)
            a=a//128
        return m
    

    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte 
        encoding for each gap
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
            (as produced by `array.tobytes` function)
        """
        ### Begin your code
        
        x=copy.deepcopy(postings_list)
        if len(postings_list)>=2:
            for i in range(len(x)):
                if i>0:
                    x[i]=postings_list[i]-postings_list[i-1]
                else:
                    x[i]=postings_list[0]
        m = bytearray()
        for i in x:
            n=CompressedPostings.switch(i)
            for j in n:
                m.append(j)
        return m
        
        
        
        
        
                
                
            
            
             
        
        

        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code
        x=list()
        y=0
        q=0
        s=0
        for i in encoded_postings_list:
            if i<128:
                y+=i*128**q
                q+=1
            else:
                i-=128
                y+=i*128**q
                s=1
            if s==1:
                x.append(y)
                s=0
                q=0
        return x
        

        ### End your code


首先，如果你实现了额外的辅助函数，写一些测试样例

In [61]:
### Begin your code
p=[31758,31886]

p.sort()
print(p)

o=CompressedPostings.encode(p)
print(o)
k=CompressedPostings.decode(o)
print(k)
### End your code

[31758, 31886]
bytearray(b'\x0ex\x81\x00\x81')
[31758, 31886]


In [62]:
p=[128*128*127]

p.sort()
print(p)

o=CompressedPostings.encode(p)
print(o)
k=CompressedPostings.decode(o)
print(k)

[2080768]
bytearray(b'\x00\x00\xff')
[2080768]


In [63]:
(127)%128

127

我们实现了一个简单的函数来测试你编码的列表是否被正确解码

In [64]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    d = CompressedPostings.decode(e)
    assert d == l
    print(l, e)

写一些测试样例来确保文档列表的压缩和解压正常运行

In [65]:
### Begin your code
p=[0, 1, 2, 303, 583, 601, 716, 746, 1020, 2008, 3172, 3396, 3397, 3571, 3678, 3716, 3729, 3852, 4042, 4043, 4298, 4299, 4301, 4701, 4868, 5614, 5680, 5919, 5983, 6448, 6471, 6557, 6574, 6763, 7250, 7837, 7952, 7954, 8005, 8222, 8695, 8854, 8869, 8894, 8923, 9389, 9401, 9631, 9975, 10304, 10354, 10421, 10431, 10435, 10448, 10449, 10450, 10514, 10530, 10570, 10578, 11595, 11599, 11702, 11767, 11801, 11977, 12093, 12095, 12099, 12101, 12102, 12103, 12104, 12105, 12106, 12107, 12201, 12673, 12705, 12715, 15387, 15751, 15812, 18542, 19114, 19673, 19675, 19676, 19799, 20126, 20127, 20134, 21504, 23284, 23290, 23294, 23296, 23297, 23300, 23304, 23306, 23308, 23311, 23313, 23315, 23323, 23344, 23644, 23667, 23965, 23973, 24018, 24027, 24030, 24031, 24040, 24063, 24064, 24130, 24133, 24136, 24139, 24140, 24146, 24152, 24206, 24207, 24221, 24227, 24228, 24229, 24231, 24247, 24257, 24260, 24268, 24319, 24322, 24323, 24324, 24330, 24332, 24333, 24335, 24336, 24339, 24340, 24341, 24345, 24359, 24364, 24369, 24374, 24375, 24376, 24377, 24383, 24389, 24391, 24393, 24407, 24408, 24410, 24413, 24425, 24426, 24436, 24446, 24447, 24452, 24466, 24480, 24486, 24499, 24500, 24506, 24521, 24549, 24551, 24558, 24561, 24565, 24570, 24574, 24978, 24985, 24988, 24999, 25008, 25009, 25012, 25013, 25120, 25270, 25548, 25570, 26710, 27270, 27285, 27348, 28014, 28171, 28212, 28259, 28448, 28727, 28731, 28732, 28759, 28760, 29424, 29435, 29450, 29732, 30030, 30195, 30521, 31486, 31758, 31886, 31887, 31888, 31893, 31894, 32222, 32231, 32243, 32559, 32683, 32684, 32731, 32860, 33000, 33013, 33717, 33739, 34226, 34351, 34580, 34826, 34827, 34940, 34959, 35235, 35304, 35389, 35478, 35579, 36139, 36214, 36522, 36932, 36933, 37415, 37418, 37419, 37420, 37422, 37425, 37430, 37433, 37603, 37604, 37605, 37606, 37607, 37608, 37609, 37610, 37611, 37612, 37613, 37614, 37615, 37616, 37617, 37618, 37619, 37620, 37621, 37622, 37623, 37624, 37625, 37626, 37627, 37628, 37629, 37630, 37631, 37632, 37633, 37634, 37635, 37636, 37637, 37638, 37639, 37640, 37641, 37642, 37643, 37644, 37645, 37646, 37811, 37994, 38123, 38202, 38219, 38221, 38692, 38699, 38822, 39114, 39117, 39138, 39145, 39172, 39194, 39272, 39347, 39357, 39358, 39359, 39360, 39437, 40545, 40591, 40652, 40655, 40656, 40662, 40665, 40666, 40797, 42037, 42078, 43000, 43492, 43801, 43803, 43842, 43844, 43964, 44290, 44293, 44295, 44301, 45177, 45203, 45217, 45218, 45221, 46141, 46322, 46326, 46378, 47115, 47137, 47138, 47161, 47385, 47387, 47390, 47394, 47396, 47397, 47398, 47453, 47471, 47605, 47610, 47645, 47646, 48321, 48322, 48337, 48525, 48526, 48530, 50512, 50521, 50902, 51153, 51169, 51461, 51463, 51464, 51504, 51520, 51545, 51546, 51547, 51549, 51551, 51567, 51569, 51614, 51615, 51616, 51639, 51642, 51652, 51654, 51684, 51729, 51774, 51798, 52272, 52273, 52274, 52275, 52276, 52277, 52455, 52511, 52597, 52605, 52608, 52912, 53078, 53421, 54064, 54509, 55245, 55310, 55311, 55495, 55758, 55778, 55797, 55829, 55830, 55863, 55883, 57484, 57499, 57501, 57521, 57538, 57551, 57555, 57560, 57575, 57607, 57608, 57614, 57619, 57625, 58019, 58027, 58375, 58436, 58523, 58686, 58874, 58915, 59052, 59411, 59543, 59566, 59767, 60130, 60131, 60334, 60398, 62587, 62588, 62592, 62593, 62596, 62599, 62602, 62605, 62608, 62609, 62611, 62612, 62620, 62622, 62623, 62626, 62627, 62631, 62632, 62635, 62637, 62645, 62646, 62648, 62649, 62652, 62654, 62657, 62663, 62669, 62670, 62675, 62700, 62701, 62702, 62704, 62717, 63003, 63013, 63439, 63693, 63725, 63727, 63928, 64124, 64159, 64206, 64777, 64778, 65171, 65479, 65540, 65546, 65547, 65722, 65734, 65735, 65739, 65743, 66241, 66326, 66419, 66445, 66446, 66448, 66452, 66454, 66468, 66476, 66717, 66721, 66724, 66890, 66908, 66930, 66958, 67609, 67657, 68254, 68335, 68340, 68343, 68351, 68352, 68353, 68357, 68381, 68382, 68415, 68416, 68417, 68418, 68434, 68436, 68441, 68442, 68443, 68449, 68451, 68454, 68461, 68469, 68471, 68474, 68500, 68502, 68504, 69150, 69151, 69152, 69153, 69154, 69155, 69156, 69157, 69158, 69159, 69160, 69161, 69162, 69163, 69164, 69165, 69166, 69167, 69168, 69169, 69170, 69171, 69172, 69173, 69174, 69175, 69176, 69177, 69178, 69179, 69180, 69181, 69182, 69183, 69184, 69185, 69186, 69187, 69188, 69189, 69190, 69191, 69192, 69193, 69194, 69195, 69196, 69197, 69198, 69199, 69200, 69201, 69202, 69203, 69204, 69205, 69206, 69207, 69208, 69209, 69210, 69211, 69212, 69213, 69214, 69215, 69216, 69217, 69218, 69219, 69220, 69221, 69222, 69223, 69224, 69225, 69226, 69227, 69228, 69229, 69346, 69437, 69707, 69717, 69718, 69719, 69720, 69721, 69722, 69723, 69724, 69725, 69726, 69727, 69728, 69729, 69730, 69731, 69864, 69870, 69949, 70093, 70176, 70181, 70206, 70207, 70251, 70257, 70259, 70260, 70263, 70617, 70618, 70727, 70729, 70736, 70769, 70798, 70847, 70860, 70866, 70870, 71116, 71193, 71199, 71708, 72164, 72167, 72170, 72172, 72181, 72214, 72215, 72216, 72218, 72235, 72345, 72346, 72347, 72348, 72407, 72419, 72420, 72458, 72459, 72461, 72462, 72469, 72474, 72482, 72484, 72494, 72504, 72507, 72508, 72512, 73727, 74074, 74602, 74717, 74765, 75984, 76057, 76230, 76469, 77188, 77215, 77216, 77223, 77230, 77279, 77300, 80688, 80863, 81153, 81237, 81605, 81794, 81898, 82357, 82359, 82394, 82395, 82428, 82429, 82443, 82586, 82587, 82649, 82695, 82980, 83425, 83530, 83771, 83772, 84006, 84111, 84279, 84292, 84434, 84604, 84605, 84626, 84656, 84658, 84725, 84871, 84872, 84961, 84962, 86099, 86103, 86105, 86591, 86885, 87872, 87945, 88461, 89945, 90153, 90450, 90458, 90459, 90533, 90538, 91143, 91146, 91150, 91423, 91640, 91646, 91647, 91716, 91733, 91948, 92038, 92595, 92601, 92706, 92824, 92860, 93091, 93299, 93430, 93431, 93437, 93439, 93443, 93445, 93455, 93470, 93493, 94386, 94582, 94583, 95406, 95433, 95452, 95563, 95587, 95637, 95968, 96596, 96750, 97507, 97653, 97671, 97674, 97712, 97744, 97758, 97761, 97763, 97989]


#print(len(p))

o=CompressedPostings.encode(p)
#print(o)
k=CompressedPostings.decode(o)

print(k==p)

"""for i in range(218,230,1):
    print(p[i])
print("\n")
for i in range(218,230,1):
    print(k[i])"""
    
    
### End your code

True


'for i in range(218,230,1):\n    print(p[i])\nprint("\n")\nfor i in range(218,230,1):\n    print(k[i])'

现在让我们创建一个新的输出文件夹并使用`CompressedPostings`来构建压缩索引

In [66]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [67]:
try: 
    os.mkdir('toy_output_dir_compressed')
except FileExistsError:
    pass

In [68]:
#玩具数据集测试
BSBI_instance_compressed1 = BSBIIndex(data_dir='toy-data', output_dir = 'toy_output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed1.index()


In [69]:
#玩具数据集测试
BSBI_instance1 = BSBIIndex(data_dir='toy-data', output_dir = 'toy_output_dir')
BSBI_instance1.index()

In [70]:
n=list()
with InvertedIndexIterator('BSBI', directory='toy_output_dir/') as k:
    for i in k:
        n.append(i)

In [71]:
l=list()
with InvertedIndexIterator('BSBI',postings_encoding=CompressedPostings ,directory='toy_output_dir_compressed/') as k:
    for i in k:
        l.append(i)

In [72]:
n==l

True

In [73]:
#正式测试部分
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
#BSBI_instance_compressed.index()

In [74]:
#正式测试的debug部分，begin
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir ='output_dir', )

In [75]:
x=list()
with InvertedIndexIterator('BSBI', directory='output_dir/') as k:
    for i in k:
        x.append(i)
   

In [76]:
y=list()
with InvertedIndexIterator('BSBI', postings_encoding=CompressedPostings,directory='output_dir_compressed/') as k:
    for i in k:
        y.append(i)

In [250]:
#BSBI_instance_compressed.retrieve("we are")

In [379]:
BSBI_instance_test1 = BSBIIndex(data_dir='pa1-data-test', output_dir = 'output_dir_test', postings_encoding=CompressedPostings)
BSBI_instance_test1.index()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



[9999]


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



[19998]


In [381]:
BSBI_instance_test2 = BSBIIndex(data_dir='pa1-data-test', output_dir = 'output_dir_test2')
BSBI_instance_test2.index()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



[9999]


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



[19998]


In [402]:
x=list()
with InvertedIndexIterator('index_0', directory='output_dir_test2/') as k:
    print(next(k))

(0, [0, 1, 2, 3716, 4868, 9975, 3852, 7952, 3729, 7954, 8854, 6557, 7837, 5919, 8222, 9631, 8869, 9389, 6574, 303, 5680, 6448, 9401, 8894, 3396, 3397, 8005, 583, 6471, 4042, 4043, 716, 4298, 4299, 4301, 7250, 2008, 601, 8923, 4701, 3678, 5983, 3172, 746, 6763, 5614, 3571, 8695, 1020])


In [398]:
y=list()
with InvertedIndexIterator('index_0', postings_encoding=CompressedPostings,directory='output_dir_test/') as k:
    for i in k:
        y.append(i)

In [81]:
BSBI_instance_test3 = BSBIIndex(data_dir='pa1-data-test', output_dir = 'output_dir_test3')
for block_dir_relative in sorted(next(os.walk(BSBI_instance_test3.data_dir))[1]):#将文件中的子目录排序
        td_pairs = BSBI_instance_test3.parse_block(block_dir_relative) #切分块
        index_id = 'index_'+block_dir_relative
        with InvertedIndexWriter(index_id, directory=BSBI_instance_test3.output_dir, 
                                 postings_encoding=
                                 BSBI_instance_test3.postings_encoding) as index:
            BSBI_instance_test3.invert_write(td_pairs, index)
            td_pairs = None
        break
BSBI_instance_test3.save()
        
       
                

In [193]:
x=list()
with InvertedIndexIterator('BSBI', directory='output_dir/') as k:
    for i in k:
        x.append(i)
#正式测试debug部分，end


像之前一样，用已构造好的查询语句来测试是否正常运行

In [80]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert set(my_results) == set(reference_results), "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


# 额外的编码方式 (10%)

通过补充`ECCompressedPostings`的`encode` 和 `decode`方法来实现一种额外的索引压缩方式。在我们课上学的就是**gamma-encoding** 。另外如果大家感兴趣的话也可以了解**Delta Encoding** ，[ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) 等。

你应该以多字节（而不是bits）来存储倒排记录表，因为索引的长度和位置都存的是字节信息。

In [208]:
import math
class ECCompressedPostings:
    
    #If you need any extra helper methods you can add them here 
    ### Begin your code

    ### End your code
    #长度(log(x)+1)+偏移原来数字的二进制去掉首位, 要求有序,将0编码为1
    @staticmethod
    def length(a):
        k=int(math.log2(a))+1
        l=2**k-2
        return l
    @staticmethod
    def gama(a):
        h=ECCompressedPostings.length(a)
        x=int(math.log2(h+1))+1
        z=2*x-1
        k=bin(h)[2:]
        m=bytearray()
        while len(k)>0:
            if len(k)>8:
                n=k[0:8]
                p=int(n,2)
                m.append(p)
                k=k[8:]
            else:
                p=int(k,2)
                m.append(p)
                k=""
        q=bin(a)[3:]
        while len(q)>0:
            if len(q)>8:
                n=q[0:8]
                p=int(n,2)
                m.append(p)
                q=q[8:]
            else:
                p=int(q,2)
                m.append(p)
                q=""
        return m
         ### End your code
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` 
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
        """
        ### Begin your code
        m=bytearray()
        for i in postings_list:
            if i==0:
                m.append(1)
                continue
            h=ECCompressedPostings.gama(i)
            for j in h:
                m.append(j)
        return m
            
        
        

        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        ### Begin your code
        m=copy.deepcopy(encoded_postings_list)
        k=""
        t=0
        u=0
        n=0
        x=list()
        o=list()
        kk=0
        if m[0]==1:
            x.append(0)
            kk+=1
            if len(m)>1:
                if m[1]==0:
                    x.append(1)
                    kk+=1
        if m[0]==0:
            x.append(1)
            kk+=1
        for i in range(kk,len(m),1):
            if t==0:
                if m[i]%2!=0:
                    k+="11111111"
                else:
                    k+=bin(m[i])[2:]
                    
                    l=len(k)
                    k=""
                    u=l-1
                    t=1
                    n=math.ceil(u/8)
                    
                    
            else:
                o.append(m[i])
                n-=1
                if n==0:
                    t=0
                    v=""
                    for i in range(len(o)):
                        if i!=len(o)-1:
                            pp=bin(o[i])[2:]
                            if len(pp)!=8:
                                for j in range(8-len(pp)):
                                    v+='0'
                            v+=pp
                        elif u%8!=0:
                            q=u%8
                            a=bin(o[i])[2:]
                            q-=len(a)
                            bb=""
                            for i in range(q):
                                v=v+'0'
                            v=v+a
                        else:
                            pp=bin(o[i])[2:]
                            if len(pp)!=8:
                                for j in range(8-len(pp)):
                                    v+='0'
                            v+=pp
                    v='1'+v
                    o=[]
                    x.append(int(v,2))
                
        return x
            
                            
                
                
                
                
                
                
                
            
                
        

        ### End your code
    

In [211]:
#代码测试
try: 
    os.mkdir('output_dir_eccompressed')
except FileExistsError:
    pass

In [212]:
try: 
    os.mkdir('toy_output_dir_eccompressed')
except FileExistsError:
    pass

In [232]:
BSBI_instance_test5 = BSBIIndex(data_dir='toy-data', output_dir = 'toy_output_dir_eccompressed',postings_encoding=ECCompressedPostings)
BSBI_instance_test5.index()

In [233]:
BSBI_instance_test4 = BSBIIndex(data_dir='toy-data', output_dir = 'toy_output_dir')
BSBI_instance_test4.index()

In [234]:
h=list()
with InvertedIndexIterator('BSBI',postings_encoding=ECCompressedPostings,directory='toy_output_dir_eccompressed/') as k:
    for i in k:
        h.append(i)

In [235]:
l=list()
with InvertedIndexIterator('BSBI',directory='toy_output_dir/') as k:
    for i in k:
        l.append(i)

In [238]:
h==l

True

In [239]:
#正式代码测试
BSBI_instance_test6 = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_eccompressed',postings_encoding=ECCompressedPostings)
BSBI_instance_test6.index()

In [241]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_test6.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert set(my_results) == set(reference_results), "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


# 作业提交

本次作业截止日期为10.8。请大家在截止日期前将代码（包含运行结果，测试内容不作要求），实验报告（可单独撰写，也可整合在jupyter notebook中）一起提交到nkulxb2022@163.com，命名方式为`学号_姓名_hw1`。